## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
from os import listdir
from sklearn.preprocessing import  LabelBinarizer
from keras.preprocessing import image
from keras_preprocessing.image import img_to_array, array_to_img
from keras.optimizers import Adam
from PIL import Image
from keras.utils import to_categorical
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,Activation, Flatten, Dropout, Dense, LeakyReLU, ReLU
from sklearn.model_selection import train_test_split

## Loading Dataset

In [2]:
# Setting path and creating empty list
dir = "Bird Speciees Dataset"
root_dir = listdir(dir)
image_list, label_list = [], []

In [3]:
for directory in root_dir:
  for files in listdir(f"{dir}/{directory}"):
    image_path = f"{dir}/{directory}/{files}"
    image = cv2.imread(image_path)
    image = img_to_array(image)
    image_list.append(image)
    label_list.append(directory)

In [4]:
# Visualize the number of classes count
label_counts = pd.DataFrame(label_list).value_counts()
label_counts

AMERICAN GOLDFINCH    143
EMPEROR PENGUIN       139
DOWNY WOODPECKER      137
FLAMINGO              132
CARMINE BEE-EATER     131
BARN OWL              129
dtype: int64

In [5]:
# Storing number of classes
num_classes = len(label_counts)
num_classes

6

In [6]:
# Checking input image shape
image_list[0].shape

(224, 224, 3)

In [7]:
# Checking labels shape 
label_list = np.array(label_list)
label_list.shape

(811,)

In [8]:
# Splitting dataset
x_train, x_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_state = 10) 

In [9]:
# Normalize and reshape data
x_train = np.array(x_train, dtype=np.float16) / 225.0
x_test = np.array(x_test, dtype=np.float16) / 225.0
x_train = x_train.reshape( -1, 224,224,3)
x_test = x_test.reshape( -1, 224,224,3)

In [10]:
# Label binarizing
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)
print(lb.classes_)

['AMERICAN GOLDFINCH' 'BARN OWL' 'CARMINE BEE-EATER' 'DOWNY WOODPECKER'
 'EMPEROR PENGUIN' 'FLAMINGO']


In [11]:
# Splitting the training data set into training and validation data sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

## Applying CNN Model

In [12]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a max pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(224,224,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(6,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 16)      208       
                                                                 
 activation (Activation)     (None, 224, 224, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 100352)            0

In [13]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [14]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
17/17 [==============================] - 7s 369ms/step - loss: 10.0265 - accuracy: 0.3475 - val_loss: 1.3104 - val_accuracy: 0.5000
Epoch 2/12
17/17 [==============================] - 7s 398ms/step - loss: 0.8564 - accuracy: 0.7220 - val_loss: 1.2898 - val_accuracy: 0.6385
Epoch 3/12
17/17 [==============================] - 6s 380ms/step - loss: 0.5375 - accuracy: 0.8069 - val_loss: 1.1526 - val_accuracy: 0.6154
Epoch 4/12
17/17 [==============================] - 8s 466ms/step - loss: 0.3937 - accuracy: 0.8861 - val_loss: 0.9013 - val_accuracy: 0.7692
Epoch 5/12
17/17 [==============================] - 7s 425ms/step - loss: 0.2355 - accuracy: 0.9266 - val_loss: 0.6153 - val_accuracy: 0.8000
Epoch 6/12
17/17 [==============================] - 8s 446ms/step - loss: 0.1809 - accuracy: 0.9459 - val_loss: 0.5802 - val_accuracy: 0.7769
Epoch 7/12
17/17 [==============================] - 7s 426ms/step - loss: 0.0579 - accuracy: 0.9923 - val_loss: 0.7878 - val_accuracy: 0.7308
Epoch

In [15]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 0s 72ms/step - loss: 0.2657 - accuracy: 0.9202
Test Accuracy: 92.02454090118408 %


## Implementing Different Activation Functions

### ELU

In [16]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a max pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(224,224,3), padding='same'))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('elu'))
model.add(Dense(6,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 224, 224, 16)      208       
                                                                 
 activation_2 (Activation)   (None, 224, 224, 16)      0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 112, 112, 16)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 32)      2080      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 100352)           

In [17]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [18]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
17/17 [==============================] - 8s 456ms/step - loss: 11.5633 - accuracy: 0.3919 - val_loss: 1.1822 - val_accuracy: 0.6308
Epoch 2/12
17/17 [==============================] - 8s 441ms/step - loss: 1.0328 - accuracy: 0.6371 - val_loss: 1.5018 - val_accuracy: 0.6000
Epoch 3/12
17/17 [==============================] - 8s 471ms/step - loss: 0.8528 - accuracy: 0.7317 - val_loss: 2.4937 - val_accuracy: 0.4231
Epoch 4/12
17/17 [==============================] - 8s 480ms/step - loss: 0.5275 - accuracy: 0.8069 - val_loss: 1.2395 - val_accuracy: 0.7231


In [19]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 83ms/step - loss: 1.3345 - accuracy: 0.7055
Test Accuracy: 70.55214643478394 %


### Selu

In [20]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a max pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(224,224,3), padding='same'))
model.add(Activation('selu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='selu', padding='same'))
model.add(MaxPooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('selu'))
model.add(Dense(6,activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 224, 224, 16)      208       
                                                                 
 activation_4 (Activation)   (None, 224, 224, 16)      0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 112, 112, 16)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 112, 112, 32)      2080      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 100352)           

In [21]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [22]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
17/17 [==============================] - 8s 460ms/step - loss: 17.7693 - accuracy: 0.3185 - val_loss: 1.8992 - val_accuracy: 0.4231
Epoch 2/12
17/17 [==============================] - 7s 441ms/step - loss: 1.6781 - accuracy: 0.4595 - val_loss: 1.3551 - val_accuracy: 0.5538
Epoch 3/12
17/17 [==============================] - 7s 434ms/step - loss: 1.6746 - accuracy: 0.5386 - val_loss: 12.9686 - val_accuracy: 0.2231
Epoch 4/12
17/17 [==============================] - 7s 436ms/step - loss: 2.2381 - accuracy: 0.6293 - val_loss: 1.5972 - val_accuracy: 0.6231
Epoch 5/12
17/17 [==============================] - 7s 437ms/step - loss: 0.5919 - accuracy: 0.7954 - val_loss: 2.2342 - val_accuracy: 0.6769


In [23]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 80ms/step - loss: 1.8707 - accuracy: 0.6810
Test Accuracy: 68.09815764427185 %


### Using Different Type of Pooling

### Average Pooling 

In [24]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a average pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(224,224,3), padding='same'))
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu', padding='same'))
model.add(AveragePooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(6,activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 224, 224, 16)      208       
                                                                 
 activation_6 (Activation)   (None, 224, 224, 16)      0         
                                                                 
 average_pooling2d (AverageP  (None, 112, 112, 16)     0         
 ooling2D)                                                       
                                                                 
 conv2d_7 (Conv2D)           (None, 112, 112, 32)      2080      
                                                                 
 average_pooling2d_1 (Averag  (None, 56, 56, 32)       0         
 ePooling2D)                                                     
                                                                 
 flatten_3 (Flatten)         (None, 100352)           

In [25]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [26]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
17/17 [==============================] - 7s 369ms/step - loss: 9.3780 - accuracy: 0.2645 - val_loss: 2.5064 - val_accuracy: 0.3000
Epoch 2/12
17/17 [==============================] - 6s 365ms/step - loss: 1.1008 - accuracy: 0.6178 - val_loss: 1.2609 - val_accuracy: 0.6000
Epoch 3/12
17/17 [==============================] - 6s 360ms/step - loss: 0.9342 - accuracy: 0.6988 - val_loss: 1.0212 - val_accuracy: 0.5538
Epoch 4/12
17/17 [==============================] - 6s 357ms/step - loss: 0.4674 - accuracy: 0.8263 - val_loss: 1.1528 - val_accuracy: 0.5231
Epoch 5/12
17/17 [==============================] - 6s 358ms/step - loss: 0.3595 - accuracy: 0.8900 - val_loss: 1.3313 - val_accuracy: 0.6154
Epoch 6/12
17/17 [==============================] - 6s 357ms/step - loss: 0.2628 - accuracy: 0.9266 - val_loss: 0.5580 - val_accuracy: 0.8231
Epoch 7/12
17/17 [==============================] - 6s 357ms/step - loss: 0.1891 - accuracy: 0.9305 - val_loss: 0.5629 - val_accuracy: 0.8462
Epoch 

In [27]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 0s 75ms/step - loss: 1.1852 - accuracy: 0.7117
Test Accuracy: 71.16564512252808 %


### Average Pooling with ELU

In [28]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a average pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(224,224,3), padding='same'))
model.add(Activation('elu'))
model.add(AveragePooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='elu', padding='same'))
model.add(AveragePooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('elu'))
model.add(Dense(6,activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 224, 224, 16)      208       
                                                                 
 activation_8 (Activation)   (None, 224, 224, 16)      0         
                                                                 
 average_pooling2d_2 (Averag  (None, 112, 112, 16)     0         
 ePooling2D)                                                     
                                                                 
 conv2d_9 (Conv2D)           (None, 112, 112, 32)      2080      
                                                                 
 average_pooling2d_3 (Averag  (None, 56, 56, 32)       0         
 ePooling2D)                                                     
                                                                 
 flatten_4 (Flatten)         (None, 100352)           

In [29]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [30]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
17/17 [==============================] - 8s 433ms/step - loss: 21.4797 - accuracy: 0.3494 - val_loss: 2.0195 - val_accuracy: 0.4615
Epoch 2/12
17/17 [==============================] - 7s 425ms/step - loss: 1.3561 - accuracy: 0.5463 - val_loss: 1.9162 - val_accuracy: 0.4231
Epoch 3/12
17/17 [==============================] - 7s 425ms/step - loss: 1.4432 - accuracy: 0.5753 - val_loss: 3.3553 - val_accuracy: 0.3846
Epoch 4/12
17/17 [==============================] - 7s 419ms/step - loss: 0.8223 - accuracy: 0.7375 - val_loss: 3.1021 - val_accuracy: 0.3000
Epoch 5/12
17/17 [==============================] - 7s 422ms/step - loss: 0.9939 - accuracy: 0.7297 - val_loss: 1.9227 - val_accuracy: 0.5231


In [31]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 120ms/step - loss: 1.3901 - accuracy: 0.5767
Test Accuracy: 57.66870975494385 %


### CNN with SELU

In [32]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a average pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(224,224,3), padding='same'))
model.add(Activation('selu'))
model.add(AveragePooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='selu', padding='same'))
model.add(AveragePooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('selu'))
model.add(Dense(6,activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 224, 224, 16)      208       
                                                                 
 activation_10 (Activation)  (None, 224, 224, 16)      0         
                                                                 
 average_pooling2d_4 (Averag  (None, 112, 112, 16)     0         
 ePooling2D)                                                     
                                                                 
 conv2d_11 (Conv2D)          (None, 112, 112, 32)      2080      
                                                                 
 average_pooling2d_5 (Averag  (None, 56, 56, 32)       0         
 ePooling2D)                                                     
                                                                 
 flatten_5 (Flatten)         (None, 100352)           

In [33]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [34]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
17/17 [==============================] - 8s 420ms/step - loss: 28.4623 - accuracy: 0.3282 - val_loss: 3.5997 - val_accuracy: 0.2231
Epoch 2/12
17/17 [==============================] - 7s 403ms/step - loss: 1.3470 - accuracy: 0.5367 - val_loss: 7.6461 - val_accuracy: 0.2077
Epoch 3/12
17/17 [==============================] - 7s 440ms/step - loss: 1.3825 - accuracy: 0.6120 - val_loss: 3.1253 - val_accuracy: 0.3615
Epoch 4/12
17/17 [==============================] - 8s 472ms/step - loss: 2.2159 - accuracy: 0.5695 - val_loss: 1.1708 - val_accuracy: 0.6385
Epoch 5/12
17/17 [==============================] - 8s 498ms/step - loss: 0.6228 - accuracy: 0.8050 - val_loss: 1.2131 - val_accuracy: 0.6923
Epoch 6/12
17/17 [==============================] - 13s 761ms/step - loss: 0.6383 - accuracy: 0.7819 - val_loss: 2.8360 - val_accuracy: 0.4692
Epoch 7/12
17/17 [==============================] - 11s 654ms/step - loss: 0.9310 - accuracy: 0.7587 - val_loss: 2.6594 - val_accuracy: 0.5769


In [35]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 198ms/step - loss: 2.2841 - accuracy: 0.5583
Test Accuracy: 55.8282196521759 %


### Increasing filters with Average Pooling

### CNN with Relu

In [48]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a average pooling layer

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=2, input_shape=(224,224,3), padding='same'))
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu', padding='same'))
model.add(AveragePooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(6,activation='softmax'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 224, 224, 32)      416       
                                                                 
 activation_18 (Activation)  (None, 224, 224, 32)      0         
                                                                 
 average_pooling2d_6 (Averag  (None, 112, 112, 32)     0         
 ePooling2D)                                                     
                                                                 
 conv2d_19 (Conv2D)          (None, 112, 112, 64)      8256      
                                                                 
 average_pooling2d_7 (Averag  (None, 56, 56, 64)       0         
 ePooling2D)                                                     
                                                                 
 flatten_9 (Flatten)         (None, 200704)           

In [49]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [50]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
17/17 [==============================] - 16s 883ms/step - loss: 8.1285 - accuracy: 0.3378 - val_loss: 1.8849 - val_accuracy: 0.3846
Epoch 2/12
17/17 [==============================] - 14s 802ms/step - loss: 0.8466 - accuracy: 0.6969 - val_loss: 1.4673 - val_accuracy: 0.5615
Epoch 3/12
17/17 [==============================] - 14s 806ms/step - loss: 0.6030 - accuracy: 0.7876 - val_loss: 0.8121 - val_accuracy: 0.7308
Epoch 4/12
17/17 [==============================] - 13s 767ms/step - loss: 0.3823 - accuracy: 0.8822 - val_loss: 1.8907 - val_accuracy: 0.5462
Epoch 5/12
17/17 [==============================] - 13s 728ms/step - loss: 0.2774 - accuracy: 0.9208 - val_loss: 2.5573 - val_accuracy: 0.5231
Epoch 6/12
17/17 [==============================] - 14s 815ms/step - loss: 0.4961 - accuracy: 0.8707 - val_loss: 0.6310 - val_accuracy: 0.8077
Epoch 7/12
17/17 [==============================] - 17s 990ms/step - loss: 0.0534 - accuracy: 0.9903 - val_loss: 0.5764 - val_accuracy: 0.8385

In [51]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 135ms/step - loss: 0.3081 - accuracy: 0.8896
Test Accuracy: 88.95705342292786 %


### CNN with ELU

In [52]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a average pooling layer

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=2, input_shape=(224,224,3), padding='same'))
model.add(Activation('elu'))
model.add(AveragePooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=2, activation='elu', padding='same'))
model.add(AveragePooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('elu'))
model.add(Dense(6,activation='softmax'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 224, 224, 32)      416       
                                                                 
 activation_20 (Activation)  (None, 224, 224, 32)      0         
                                                                 
 average_pooling2d_8 (Averag  (None, 112, 112, 32)     0         
 ePooling2D)                                                     
                                                                 
 conv2d_21 (Conv2D)          (None, 112, 112, 64)      8256      
                                                                 
 average_pooling2d_9 (Averag  (None, 56, 56, 64)       0         
 ePooling2D)                                                     
                                                                 
 flatten_10 (Flatten)        (None, 200704)          

In [53]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [54]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
17/17 [==============================] - 17s 957ms/step - loss: 29.9404 - accuracy: 0.2625 - val_loss: 2.8329 - val_accuracy: 0.3154
Epoch 2/12
17/17 [==============================] - 16s 958ms/step - loss: 1.1207 - accuracy: 0.6274 - val_loss: 1.8529 - val_accuracy: 0.5000
Epoch 3/12
17/17 [==============================] - 16s 949ms/step - loss: 2.1116 - accuracy: 0.5656 - val_loss: 3.0458 - val_accuracy: 0.4462
Epoch 4/12
17/17 [==============================] - 16s 951ms/step - loss: 0.8287 - accuracy: 0.7259 - val_loss: 7.4113 - val_accuracy: 0.2692
Epoch 5/12
17/17 [==============================] - 15s 851ms/step - loss: 1.8181 - accuracy: 0.6564 - val_loss: 0.9587 - val_accuracy: 0.7231
Epoch 6/12
17/17 [==============================] - 14s 799ms/step - loss: 0.7648 - accuracy: 0.7529 - val_loss: 3.0476 - val_accuracy: 0.5308
Epoch 7/12
17/17 [==============================] - 16s 954ms/step - loss: 1.2502 - accuracy: 0.7471 - val_loss: 9.3249 - val_accuracy: 0.446

In [55]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 2s 277ms/step - loss: 1.1270 - accuracy: 0.7301
Test Accuracy: 73.00613522529602 %


### CNN with SELU

In [56]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a average pooling layer

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=2, input_shape=(224,224,3), padding='same'))
model.add(Activation('selu'))
model.add(AveragePooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=2, activation='selu', padding='same'))
model.add(AveragePooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('selu'))
model.add(Dense(6,activation='softmax'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 224, 224, 32)      416       
                                                                 
 activation_22 (Activation)  (None, 224, 224, 32)      0         
                                                                 
 average_pooling2d_10 (Avera  (None, 112, 112, 32)     0         
 gePooling2D)                                                    
                                                                 
 conv2d_23 (Conv2D)          (None, 112, 112, 64)      8256      
                                                                 
 average_pooling2d_11 (Avera  (None, 56, 56, 64)       0         
 gePooling2D)                                                    
                                                                 
 flatten_11 (Flatten)        (None, 200704)          

In [57]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [58]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
17/17 [==============================] - 18s 1s/step - loss: 71.9287 - accuracy: 0.2664 - val_loss: 4.8356 - val_accuracy: 0.3615
Epoch 2/12
17/17 [==============================] - 17s 1s/step - loss: 3.3575 - accuracy: 0.4981 - val_loss: 4.9618 - val_accuracy: 0.4154
Epoch 3/12
17/17 [==============================] - 17s 1000ms/step - loss: 2.1236 - accuracy: 0.5463 - val_loss: 4.3423 - val_accuracy: 0.4692
Epoch 4/12
17/17 [==============================] - 20s 1s/step - loss: 6.2161 - accuracy: 0.4865 - val_loss: 2.7284 - val_accuracy: 0.5846
Epoch 5/12
17/17 [==============================] - 20s 1s/step - loss: 0.8937 - accuracy: 0.7664 - val_loss: 2.2236 - val_accuracy: 0.6231
Epoch 6/12
17/17 [==============================] - 18s 1s/step - loss: 2.2373 - accuracy: 0.6660 - val_loss: 6.1287 - val_accuracy: 0.3769
Epoch 7/12
17/17 [==============================] - 17s 1s/step - loss: 1.4678 - accuracy: 0.7278 - val_loss: 2.5593 - val_accuracy: 0.6308
Epoch 8/12
17/1

In [59]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 2s 258ms/step - loss: 4.2831 - accuracy: 0.5583
Test Accuracy: 55.8282196521759 %


## Using different Architecture

### LeNet

In [60]:
model = Sequential()

model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(224,224,3)))
model.add(AveragePooling2D())

model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())

model.add(Flatten())
model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=84, activation='relu'))
model.add(Dense(6, activation = 'softmax'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 222, 222, 6)       168       
                                                                 
 average_pooling2d_12 (Avera  (None, 111, 111, 6)      0         
 gePooling2D)                                                    
                                                                 
 conv2d_25 (Conv2D)          (None, 109, 109, 16)      880       
                                                                 
 average_pooling2d_13 (Avera  (None, 54, 54, 16)       0         
 gePooling2D)                                                    
                                                                 
 flatten_12 (Flatten)        (None, 46656)             0         
                                                                 
 dense_24 (Dense)            (None, 120)             

In [61]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['accuracy'])

In [62]:
# Training the model
epochs = 50
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/50
17/17 [==============================] - 8s 354ms/step - loss: 1.7804 - accuracy: 0.3533 - val_loss: 1.3677 - val_accuracy: 0.5077
Epoch 2/50
17/17 [==============================] - 6s 376ms/step - loss: 0.9782 - accuracy: 0.6660 - val_loss: 1.0139 - val_accuracy: 0.5923
Epoch 3/50
17/17 [==============================] - 6s 339ms/step - loss: 0.6168 - accuracy: 0.8089 - val_loss: 0.6910 - val_accuracy: 0.7538
Epoch 4/50
17/17 [==============================] - 6s 380ms/step - loss: 0.3421 - accuracy: 0.8938 - val_loss: 0.6119 - val_accuracy: 0.8077
Epoch 5/50
17/17 [==============================] - 6s 359ms/step - loss: 0.2110 - accuracy: 0.9479 - val_loss: 0.5831 - val_accuracy: 0.7769
Epoch 6/50
17/17 [==============================] - 5s 306ms/step - loss: 0.1414 - accuracy: 0.9595 - val_loss: 0.5359 - val_accuracy: 0.8154
Epoch 7/50
17/17 [==============================] - 5s 300ms/step - loss: 0.0947 - accuracy: 0.9768 - val_loss: 0.6249 - val_accuracy: 0.7923
Epoch 

In [63]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 0s 76ms/step - loss: 0.4404 - accuracy: 0.8344
Test Accuracy: 83.43558311462402 %


### AlexNet

In [64]:
model = Sequential()

model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation="relu", input_shape=(224,224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides= (2, 2)))

model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(6, activation="softmax"))

In [65]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['accuracy'])

In [66]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
17/17 [==============================] - 26s 1s/step - loss: 24.2003 - accuracy: 0.3320 - val_loss: 2075.7200 - val_accuracy: 0.1692
Epoch 2/12
17/17 [==============================] - 33s 2s/step - loss: 9.9889 - accuracy: 0.5077 - val_loss: 459.0771 - val_accuracy: 0.1923
Epoch 3/12
17/17 [==============================] - 29s 2s/step - loss: 7.4328 - accuracy: 0.6042 - val_loss: 225.8392 - val_accuracy: 0.1923
Epoch 4/12
17/17 [==============================] - 27s 2s/step - loss: 4.9310 - accuracy: 0.6216 - val_loss: 67.2606 - val_accuracy: 0.3308
Epoch 5/12
17/17 [==============================] - 25s 2s/step - loss: 6.7061 - accuracy: 0.6448 - val_loss: 111.9508 - val_accuracy: 0.2000
Epoch 6/12
17/17 [==============================] - 26s 2s/step - loss: 7.2465 - accuracy: 0.6757 - val_loss: 48.4925 - val_accuracy: 0.2923
Epoch 7/12
17/17 [==============================] - 29s 2s/step - loss: 5.3859 - accuracy: 0.6737 - val_loss: 25.9040 - val_accuracy: 0.2692
Epoch 8

In [67]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 2s 296ms/step - loss: 3.3974 - accuracy: 0.6748
Test Accuracy: 67.48466491699219 %
